# Experiment 037: Truly Asymmetric Random Initialization + SA

Generate completely random configurations from scratch (not perturbing baseline, not grid-based):
1. Random positions in a large box
2. Random angles (0-360°, not just 0°, 45°, 90°)
3. Run SA to optimize
4. Compare with baseline

Focus on N=10-50 where efficiency is 66-72%.

In [2]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import random
import time
import math

# Tree shape
TREE_VERTICES = np.array([
    [0.0, 0.8], [0.125, 0.5], [0.0625, 0.5], [0.2, 0.25], [0.1, 0.25],
    [0.35, 0.0], [0.075, 0.0], [0.075, -0.2], [-0.075, -0.2], [-0.075, 0.0],
    [-0.35, 0.0], [-0.1, 0.25], [-0.2, 0.25], [-0.0625, 0.5], [-0.125, 0.5],
], dtype=np.float64)

def create_tree_polygon(x, y, deg):
    tree = Polygon(TREE_VERTICES)
    tree = rotate(tree, deg, origin=(0, 0))
    tree = translate(tree, x, y)
    return tree

def check_overlap(trees):
    n = len(trees)
    for i in range(n):
        for j in range(i + 1, n):
            if trees[i].overlaps(trees[j]) or trees[i].contains(trees[j]) or trees[j].contains(trees[i]):
                return True
    return False

def get_bounding_box_side(trees):
    all_bounds = [t.bounds for t in trees]
    min_x = min(b[0] for b in all_bounds)
    min_y = min(b[1] for b in all_bounds)
    max_x = max(b[2] for b in all_bounds)
    max_y = max(b[3] for b in all_bounds)
    return max(max_x - min_x, max_y - min_y)

def calculate_score(trees):
    side = get_bounding_box_side(trees)
    return side * side / len(trees)

def parse_value(v):
    if isinstance(v, str) and v.startswith('s'):
        return float(v[1:])
    return float(v)

print("Functions defined")

Functions defined


In [3]:
# Load baseline
df = pd.read_csv('/home/submission/submission.csv')

baseline_configs = {}
baseline_scores = {}

for n in range(1, 201):
    prefix = f"{n:03d}_"
    group = df[df["id"].str.startswith(prefix)].sort_values("id")
    configs = []
    for _, row in group.iterrows():
        x = parse_value(row["x"])
        y = parse_value(row["y"])
        deg = parse_value(row["deg"])
        configs.append((x, y, deg))
    baseline_configs[n] = configs
    trees = [create_tree_polygon(x, y, deg) for x, y, deg in configs]
    baseline_scores[n] = calculate_score(trees)

print(f"Baseline total: {sum(baseline_scores.values()):.6f}")
print(f"\nBaseline scores for N=10-50:")
for n in [10, 20, 30, 40, 50]:
    print(f"  N={n}: {baseline_scores[n]:.6f}")

Baseline total: 70.626088

Baseline scores for N=10-50:
  N=10: 0.376630
  N=20: 0.376057
  N=30: 0.360883
  N=40: 0.362148
  N=50: 0.360753


In [4]:
def generate_random_config(n, box_size=10.0):
    """Generate completely random configuration."""
    configs = []
    for i in range(n):
        x = random.uniform(-box_size/2, box_size/2)
        y = random.uniform(-box_size/2, box_size/2)
        deg = random.uniform(0, 360)  # Truly random angle
        configs.append((x, y, deg))
    return configs

def sa_optimize(configs, max_iter=10000, T_start=1.0, T_end=0.001):
    """Simulated annealing optimization."""
    n = len(configs)
    current = list(configs)
    
    # Evaluate initial
    trees = [create_tree_polygon(x, y, deg) for x, y, deg in current]
    if check_overlap(trees):
        current_score = float('inf')
    else:
        current_score = calculate_score(trees)
    
    best = list(current)
    best_score = current_score
    
    # Temperature schedule
    alpha = (T_end / T_start) ** (1.0 / max_iter)
    T = T_start
    
    for iteration in range(max_iter):
        # Generate neighbor
        new = list(current)
        idx = random.randint(0, n - 1)
        x, y, deg = new[idx]
        
        move_type = random.randint(0, 2)
        if move_type == 0:  # Move position
            x += random.gauss(0, 0.1 * T / T_start + 0.01)
            y += random.gauss(0, 0.1 * T / T_start + 0.01)
        elif move_type == 1:  # Rotate
            deg += random.gauss(0, 10 * T / T_start + 1)
            deg = deg % 360
        else:  # Both
            x += random.gauss(0, 0.05 * T / T_start + 0.005)
            y += random.gauss(0, 0.05 * T / T_start + 0.005)
            deg += random.gauss(0, 5 * T / T_start + 0.5)
            deg = deg % 360
        
        new[idx] = (x, y, deg)
        
        # Evaluate
        trees = [create_tree_polygon(x, y, deg) for x, y, deg in new]
        if check_overlap(trees):
            new_score = float('inf')
        else:
            new_score = calculate_score(trees)
        
        # Accept or reject
        delta = new_score - current_score
        if delta < 0 or (current_score < float('inf') and random.random() < math.exp(-delta / T)):
            current = new
            current_score = new_score
            
            if new_score < best_score:
                best = list(new)
                best_score = new_score
        
        T *= alpha
    
    return best, best_score

print("SA optimizer defined")

SA optimizer defined


In [5]:
# Run truly asymmetric random initialization + SA for N=10-50
print("Running truly asymmetric random initialization + SA for N=10-50...")
print("(50 random restarts per N, 10000 SA iterations each)")
print()

results = {}
improvements = []

for n in range(10, 51):
    t0 = time.time()
    best_config = None
    best_score = float('inf')
    
    # Multiple random restarts
    for restart in range(50):
        # Generate truly random config
        config = generate_random_config(n, box_size=n * 0.5)  # Scale box with N
        
        # Optimize with SA
        optimized, score = sa_optimize(config, max_iter=10000)
        
        if score < best_score:
            best_score = score
            best_config = optimized
    
    elapsed = time.time() - t0
    results[n] = (best_config, best_score)
    
    improvement = baseline_scores[n] - best_score
    if improvement > 0:
        improvements.append((n, improvement))
        print(f"N={n}: IMPROVED! {best_score:.6f} vs baseline {baseline_scores[n]:.6f} (improvement: {improvement:+.6f}) ({elapsed:.1f}s)")
    elif n % 10 == 0:
        print(f"N={n}: {best_score:.6f} vs baseline {baseline_scores[n]:.6f} (improvement: {improvement:+.6f}) ({elapsed:.1f}s)")

print(f"\nTotal improvements found: {len(improvements)}")

Running truly asymmetric random initialization + SA for N=10-50...
(50 random restarts per N, 10000 SA iterations each)



N=10: 1.425095 vs baseline 0.376630 (improvement: -1.048465) (321.0s)


N=20: 3.771717 vs baseline 0.376057 (improvement: -3.395660) (795.2s)


N=30: 7.551989 vs baseline 0.360883 (improvement: -7.191105) (1141.0s)


N=40: 9.224799 vs baseline 0.362148 (improvement: -8.862651) (2187.6s)


N=50: 11.588670 vs baseline 0.360753 (improvement: -11.227917) (2585.7s)

Total improvements found: 0


In [ ]:
# Summary
print("\n" + "="*60)
print("EXPERIMENT 037 SUMMARY: Truly Asymmetric Random Init + SA")
print("="*60)

total_improvement = sum(imp for _, imp in improvements) if improvements else 0
print(f"\nTotal improvements found: {len(improvements)}")
print(f"Total improvement: {total_improvement:.6f}")

if improvements:
    print("\nN values improved:")
    for n, imp in improvements:
        print(f"  N={n}: {imp:+.6f}")
else:
    print("\nNo improvements found - baseline is already optimal for N=10-50")
    print("Random asymmetric configurations + SA cannot beat the baseline.")

In [ ]:
# Save metrics
import json

final_score = 70.626088  # No improvement expected

metrics = {
    'cv_score': final_score,
    'baseline_score': 70.626088,
    'improvement': total_improvement,
    'improvements_found': len(improvements),
    'n_range': '10-50',
    'restarts_per_n': 50,
    'sa_iterations': 10000,
    'approach': 'Truly asymmetric random initialization + SA for N=10-50',
    'conclusion': 'No improvements found' if not improvements else f'Found {len(improvements)} improvements'
}

with open('/home/code/experiments/037_truly_asymmetric_random/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("\nMetrics saved:")
for k, v in metrics.items():
    print(f"  {k}: {v}")